In [1]:
from scipy.sparse import load_npz

In [2]:
train = load_npz('../data/preprocessed/1-gram/train.npz')
train.shape

(69932, 5863)

### Features

- 0
  - skupina bodového hodnotenia
  - ordinálny atribút
  - Z+
- 1
  - hodnotenie
  - numericky spojitý atribút
  - <0, 1>
- 2
  - cena
  - numericky spojitý atribút
  - <0, 1>
- 3
  - hodnotenie ^ 2
  - numericky spojitý atribút
  - <0, 1>
- 4
  - cena ^ 2
  - numericky spojitý atribút
  - <0, 1>
- 5
  - hodnotenie * cena
  - numericky spojitý atribút
  - <0, 1>
- 6 - 46 (41 atribútov)
  - krajina pôvodu
  - nominálny atribút
  - \[0, 1]
- 47 - 65 (19 atribútov)
  - priateľ vína
  - nominálny atribút
  - \[0, 1]
- 66 - 5862
  - recenzia
  - tf-idf (spojitý)
  - <0, 1>

##### Disclaimer
Niektoré atribúty sú tie isté alebo navzájom závislé, a preto je potrebné zvážiť ich výber a použitie.
